In [18]:
import cv2
import numpy as np

def project_bbox_homography(img1, img2, bbox1):
    # bbox1 = (x, y, w, h) in img1
    x,y,w,h = bbox1
    src_corners = np.float32([[x,y],[x+w,y],[x+w,y+h],[x,y+h]]).reshape(-1,1,2)

    # 1) features & matches
    orb = cv2.ORB_create(4000)
    k1,d1 = orb.detectAndCompute(img1, None)
    k2,d2 = orb.detectAndCompute(img2, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    matches = bf.knnMatch(d1, d2, k=2)

    # Lowe ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
    # print(good)
    if len(good) < 1:
        raise RuntimeError("Not enough matches for robust H")

    pts1 = np.float32([k1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    pts2 = np.float32([k2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

    # 2) RANSAC homography
    H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 3.0)
    if H is None:
        raise RuntimeError("Homography failed")

    # 3) project box corners
    dst = cv2.perspectiveTransform(src_corners, H).reshape(-1,2)

    # Optional: axis-aligned bbox
    xs, ys = dst[:,0], dst[:,1]
    x2, y2 = float(xs.min()), float(ys.min())
    w2, h2 = float(xs.max()-xs.min()), float(ys.max()-ys.min())
    return {
    "aabb2": (x2, y2, w2, h2),
    "quad2": dst,
    "H": H,
}


In [19]:
import cv2
import numpy as np
from typing import Tuple

def draw_projection_viz(
    img2: np.ndarray,
    quad2: np.ndarray,
    aabb2: Tuple[float, float, float, float],
    alpha_quad: float = 0.35
) -> np.ndarray:
    """
    Draw the projected quadrilateral (green) and its AABB (red) on a copy of img2.
    quad2: array-like shape (4,2) of projected corners in img2.
    aabb2: (x, y, w, h) in img2 coords.
    """
    vis = img2.copy()

    # Quad outline + translucent fill
    poly = quad2.reshape(4, 1, 2).astype(np.int32)
    cv2.polylines(vis, [poly], isClosed=True, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    overlay = vis.copy()
    cv2.fillPoly(overlay, [poly], color=(0, 255, 0))
    vis = cv2.addWeighted(overlay, alpha_quad, vis, 1 - alpha_quad, 0)

    # Axis-aligned bounding box
    x, y, w, h = aabb2
    p1 = (int(round(x)), int(round(y)))
    p2 = (int(round(x + w)), int(round(y + h)))
    cv2.rectangle(vis, p1, p2, (0, 0, 255), 2, lineType=cv2.LINE_AA)

    return vis


In [20]:
import json
import os
def load_annotations_for_frame(ann_folder, frame_index):
    # print(ann_folder)
    ann_file = os.path.join(ann_folder, "frame_" + f"{frame_index:06d}.json")
    # print(ann_file)
    if not os.path.exists(ann_file):
        # print("here")
        return []

    with open(ann_file, 'r') as f:
        data = json.load(f)

    bboxes = []
    hand_ids = []
    centers = []
    for ann in data.get("annotations", []):
        bbox = ann.get("bbox", [])
        hand_ids.append(ann.get("category_id"))
        if(len(bbox)==4):
            bboxes.append(bbox)
            x, y, w, h = bbox
            cx = x + w / 2.0
            cy = y + h / 2.0
            center = [cx,cy]
            centers.append(center)
        # else:
            # print("HERE HEREH EHREHREHREHREHR EHRHE RHE RHEHR EHR EHRHERHEHREHREHR HERH EHR HERHEHREHR EH")
    return bboxes, centers, hand_ids

In [21]:
def find_center(bbox):
    x, y, w, h = bbox
    cx = x + w / 2.0
    cy = y + h / 2.0
    center = (cx,cy)
    return center

In [22]:
# import cv2
# import os
# from pathlib import Path
# for dir in dirs: 
#     root_dir = "/Users/eddie/Trauma_THOMPSON/Trauma_THOMPSON/Hand Tracking Dataset/Hand Tracking Dataset/train/" + dir + "/" + dir + "/"
#     bboxes = []
#     hand_ids = []
#     centers = []
#     for index in range(1, len(os.listdir(root_dir))):
#         bbox, center, hand_id = load_annotations_for_frame(root_dir, index)
#         bboxes.append(bbox)
#         hand_ids.append(hand_id)
#         centers.append(center)
# # annotations = os.listdir(root_dir)
# # for annotation in annotations[:100]:
# #     path = Path(root_dir+annotation)
# #     with path.open("r", encoding="utf-8") as f:
# #         data = json.load(f)
# #     print(annotation,data["annotations"])


In [23]:
# print(bboxes)
# print(len(bboxes))

In [24]:
# print(hand_ids)
# print(len(hand_ids))

In [25]:
# num_frames = 30
# frames_root_dir = "/Users/eddie/Downloads/temp/P02_38/"
# projected_centers = []

# for index in range(len(bboxes)):
#     temp_centers = []
#     frame1 = cv2.imread(frames_root_dir+"frame_"+f"{index+1:06d}.jpg")
#     next_index = index+1
#     while next_index<len(bboxes) and next_index-index<=num_frames:
#         next_center = []
#         for bbox in bboxes[index]:
#             frame2 = cv2.imread(frames_root_dir+"frame_"+f"{next_index+1:06d}.jpg")
#             # print(frames_root_dir+"frame_"+f"{next_index+1:06d}.jpg")
#             res = project_bbox_homography(frame1, frame2, bbox)
#             next_center.append(res["aabb2"])
#         print(next_index, next_center)    
#         temp_centers.append(next_center)
#         next_index = next_index+1
#     projected_centers.append(temp_centers)
    
    

In [26]:
# print(projected_centers[0][0][0][0])
# print(len(projected_centers))

In [27]:
# def find_center(bbox):
#     x, y, w, h = bbox
#     cx = x + w / 2.0
#     cy = y + h / 2.0
#     center = (cx,cy)
#     return center

In [28]:
# import csv
# import pandas as pd 

# rows = [["frame", "left_hand", "right_hand", "left_bbox", "right_bbox"]]
# for j in range(len(projected_centers)):
#     left_row = []
#     right_row = []
#     left_row_center = []
#     right_row_center = []
#     for i in range(len(projected_centers[j])):
#         # print(len(centers[i]))
#         if len(projected_centers[j][i]) == 0:
#             left_hand = (10000, 10000, 10000, 10000)
#             left_center = 10000
#             right_hand = (10000, 10000, 10000, 10000)
#             right_center = 10000
#         elif len(projected_centers[j][i])==1 and hand_ids[i][0]==0:
#             left_hand = projected_centers[j][i][0]
#             left_center = find_center(projected_centers[j][i][0])
#             right_hand= (10000, 10000, 10000, 10000)
#             right_center=10000
#         elif len(projected_centers[j][i])==1 and hand_ids[i][0]==1:
#             left_hand = (10000, 10000, 10000, 10000)
#             left_center = 10000
#             right_hand = projected_centers[j][i][0]
#             right_center = find_center(projected_centers[j][i][0])
#         else:
#             left_hand = projected_centers[j][i][0]
#             left_center = find_center(projected_centers[j][i][0])
#             right_hand = projected_centers[j][i][1]
#             right_center = find_center(projected_centers[j][i][1])
#         left_row.append(left_hand)
#         right_row.append(right_hand)
#         left_row_center.append(left_center)
#         right_row_center.append(right_center)
#         print(left_row, right_row)
#     rows.append(["frame_"+f"{j+1:06d}",left_row_center, right_row_center, left_row, right_row])
# with open("P02_38.csv", "w", newline="", encoding="utf-8") as f:
#     writer = csv.writer(f)
#     writer.writerows(rows)

In [29]:
import cv2
import os
from pathlib import Path
# print(os.listdir("/Users/eddie/Trauma_THOMPSON/Trauma_THOMPSON/Hand Tracking Dataset/Hand Tracking Dataset/train/"))
dirs = os.listdir("/Users/eddie/Trauma_THOMPSON/Trauma_THOMPSON/Hand Tracking Dataset/Hand Tracking Dataset/test/")
for dir in dirs[6:]: 
    root_dir = "/Users/eddie/Trauma_THOMPSON/Trauma_THOMPSON/Hand Tracking Dataset/Hand Tracking Dataset/test/" + dir + "/" + dir + "/"
    bboxes = []
    hand_ids = []
    centers = []
    # print(dir, len(os.listdir(root_dir)))
    for index in range(len(os.listdir(root_dir))):
        # print(index)
        bbox, center, hand_id = load_annotations_for_frame(root_dir, index)
        bboxes.append(bbox)
        hand_ids.append(hand_id)
        centers.append(center)
    # print(bboxes)
    # print(len(bboxes))
    # print(hand_ids)
    # print(len(hand_ids))
    num_frames = 10
    frames_root_dir = "/Users/eddie/Downloads/temp/" + dir + "/"
    projected_centers = []

    for index in range(len(bboxes)):
        temp_centers = []
        frame1 = cv2.imread(frames_root_dir+"frame_"+f"{index+1:06d}.jpg")
        # print(index)
        next_index = index+1
        while next_index<len(bboxes) and next_index-index<=num_frames:
            # print("here")
            next_center = []
            for bbox in bboxes[index]:
                frame2 = cv2.imread(frames_root_dir+"frame_"+f"{next_index+1:06d}.jpg")
                print(frames_root_dir+"frame_"+f"{next_index+1:06d}.jpg")
                res = project_bbox_homography(frame1, frame2, bbox)
                next_center.append(res["aabb2"])
            # print(next_index, next_center)    
            temp_centers.append(next_center)
            next_index = next_index+1
        projected_centers.append(temp_centers)
    # print(projected_centers[0][0][0][0])
    # print(len(projected_centers))
    import csv
    import pandas as pd 

    rows = [["frame", "left_hand", "right_hand", "left_bbox", "right_bbox"]]
    for j in range(len(projected_centers)):
        left_row = []
        right_row = []
        left_row_center = []
        right_row_center = []
        # print(j)
        for i in range(len(projected_centers[j])):
            # print(len(centers[i]))
            if len(projected_centers[j][i]) == 0:
                left_hand = (10000, 10000, 10000, 10000)
                left_center = 10000
                right_hand = (10000, 10000, 10000, 10000)
                right_center = 10000
            elif len(projected_centers[j][i])==1 and hand_ids[j][0]==0:
                left_hand = projected_centers[j][i][0]
                left_center = find_center(projected_centers[j][i][0])
                right_hand= (10000, 10000, 10000, 10000)
                right_center=10000
            elif len(projected_centers[j][i])==1 and hand_ids[j][0]==1:
                left_hand = (10000, 10000, 10000, 10000)
                left_center = 10000
                right_hand = projected_centers[j][i][0]
                right_center = find_center(projected_centers[j][i][0])
            else:
                left_hand = projected_centers[j][i][0]
                left_center = find_center(projected_centers[j][i][0])
                right_hand = projected_centers[j][i][1]
                right_center = find_center(projected_centers[j][i][1])
            left_row.append(left_hand)
            right_row.append(right_hand)
            left_row_center.append(left_center)
            right_row_center.append(right_center)
            # print(left_row, right_row)
        rows.append(["frame_"+f"{j+1:06d}",left_row_center, right_row_center, left_row, right_row])
    with open(dir+"_10_test.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(rows)
        
# annotations = os.listdir(root_dir)
# for annotation in annotations[:100]:
#     path = Path(root_dir+annotation)
#     with path.open("r", encoding="utf-8") as f:
#         data = json.load(f)
#     print(annotation,data["annotations"])


/Users/eddie/Downloads/temp/P02_10/frame_000135.jpg
/Users/eddie/Downloads/temp/P02_10/frame_000136.jpg
/Users/eddie/Downloads/temp/P02_10/frame_000137.jpg
/Users/eddie/Downloads/temp/P02_10/frame_000138.jpg
/Users/eddie/Downloads/temp/P02_10/frame_000139.jpg
/Users/eddie/Downloads/temp/P02_10/frame_000140.jpg
/Users/eddie/Downloads/temp/P02_10/frame_000141.jpg
/Users/eddie/Downloads/temp/P02_10/frame_000142.jpg


RuntimeError: Homography failed

In [ ]:
    # # Load images
    # img1 = cv2.imread("frame-1.png")
    # img2 = cv2.imread("frame-2.png")

    # import cv2

    # # Show raw images
    # cv2.imshow("Image 1", img1)
    # cv2.imshow("Image 2", img2)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [ ]:


# # Bounding box in img1 (x, y, w, h)
# bbox1 = (340, 256, 258, 311)

# res = project_bbox_homography(img1, img2, bbox1)
# print(res["aabb2"])
# aabb_int = tuple(int(round(v)) for v in res["aabb2"])
# # viz = draw_projection_viz(img2, res["quad2"], res["aabb2"])
# # cv2.imshow("Projected Box", viz)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# def draw_bbox_safe(img, bbox, color=(0,255,0), thickness=2, title="bbox"):
#     """
#     Draw an axis-aligned bbox on img with lots of sanity checks.
#     bbox can be (x, y, w, h) or (x1, y1, x2, y2) (auto-detected).
#     Returns a copy with the box drawn. Also displays it (OpenCV with matplotlib fallback).
#     """
#     if img is None:
#         raise ValueError("Image is None (failed to load).")
#     if not hasattr(img, "shape") or img.size == 0:
#         raise ValueError("Image is empty (size==0).")

#     H, W = img.shape[:2]

#     # ensure color image so the box is visible
#     vis = img
#     if img.ndim == 2:
#         vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

#     # ensure type is uint8 0..255
#     if vis.dtype != np.uint8:
#         vmin, vmax = float(vis.min()), float(vis.max())
#         if vmax <= 1.0:  # likely 0..1 floats
#             vis = np.clip(vis * 255.0, 0, 255).astype(np.uint8)
#         else:
#             vis = np.clip(vis, 0, 255).astype(np.uint8)

#     bbox = np.array(bbox, dtype=float).ravel()
#     if bbox.size != 4 or not np.isfinite(bbox).all():
#         raise ValueError(f"Bad bbox: {bbox}")

#     # auto-detect format: (x,y,w,h) vs (x1,y1,x2,y2)
#     x, y, w, h = bbox
#     if w > 0 and h > 0 and (x + w > x) and (y + h > y):
#         x1, y1, x2, y2 = x, y, x + w, y + h
#     else:
#         # assume (x1,y1,x2,y2)
#         x1, y1, x2, y2 = bbox
#         # fix inverted corners if needed
#         if x2 < x1: x1, x2 = x2, x1
#         if y2 < y1: y1, y2 = y2, y1

#     # clip to image bounds (and report if fully offscreen)
#     if x2 < 0 or y2 < 0 or x1 >= W or y1 >= H:
#         raise ValueError(f"BBox is fully outside image: {(x1,y1,x2,y2)} vs image size {(W,H)}")

#     x1c = int(np.clip(round(x1), 0, W-1))
#     y1c = int(np.clip(round(y1), 0, H-1))
#     x2c = int(np.clip(round(x2), 0, W-1))
#     y2c = int(np.clip(round(y2), 0, H-1))

#     if x2c == x1c: x2c = min(x1c+1, W-1)
#     if y2c == y1c: y2c = min(y1c+1, H-1)

#     # scale thickness for big images
#     t = max(thickness, int(round(0.003 * max(H, W))))
#     cv2.rectangle(vis, (x1c, y1c), (x2c, y2c), color, t, lineType=cv2.LINE_AA)

#     # corner dots help visibility
#     r = max(3, int(round(0.006 * max(H, W))))
#     for (px, py) in [(x1c,y1c),(x2c,y1c),(x2c,y2c),(x1c,y2c)]:
#         cv2.circle(vis, (px,py), r, color, -1, lineType=cv2.LINE_AA)

#     # show (with headless fallback)
#     try:
#         cv2.imshow(title, vis); cv2.waitKey(0); cv2.destroyAllWindows()
#     except Exception:
#         import matplotlib.pyplot as plt
#         plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)); plt.axis("off"); plt.title(title); plt.show()

#     return vis
# img = cv2.imread("frame-1.png")
# bbox =  projected_centers[0][9][0]
# print(bbox)
# out = draw_bbox_safe(img, bbox, color=(0,255,0), thickness=2, title="check bbox")
